In [15]:
import os
from dotenv import load_dotenv
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.utils import formatdate
from agents import function_tool

load_dotenv(override=True)

smtp_configuration = {
    "server": os.getenv("SMTP_SERVER"),
    "port": os.getenv("SMTP_PORT"),
    "username": os.getenv("SMTP_USERNAME"),
    "password": os.getenv("SMTP_PASSWORD"),
    "from": os.getenv("SMTP_SENDER_EMAIL")
}


@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """

    msg = MIMEMultipart()
    msg['From'] = smtp_configuration["from"]
    msg['To'] = "nobelnurulislam@gmail.com"
    msg['Subject'] = "Sales email"
    msg['Date'] = formatdate(localtime=True)

       # Attach body
    msg.attach(MIMEText(body, 'plain'))

    with smtplib.SMTP_SSL(smtp_configuration["server"], smtp_configuration["port"]) as server:
            server.login(smtp_configuration["username"], smtp_configuration["password"])
            # Combine all recipients
            recipients = ["nobelnurulislam@gmail.com"]
            server.sendmail(smtp_configuration["from"], recipients, msg.as_string())
            
    return {"status": "success"}


In [23]:
from agents import Agent


instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
)
tool1 = sales_agent1.as_tool("sales_agent1", instructions1)

sales_agent2 = Agent(
    name="Humorous Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
)
tool2 = sales_agent2.as_tool("sales_agent2", instructions2)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
)
tool3 = sales_agent3.as_tool("sales_agent3", instructions3)

tools = [tool1, tool2, tool3, send_email]


In [24]:
from agents import Runner, trace


sales_manager = Agent(
    name="Sales Manager",
    instructions="You are a sales manager for ComplAI, \
        you use the tools provided to you to generate cold sales emails to potential customers.\
        you only use the tools provided to you to generate the emails.\
        you pick the best email from the emails generated by sales agents tools.\
        Then you send the email to the customer using the send_email tool.",
    tools=tools,
    model="gpt-4o-mini"
)



message = "Send a cold sales email addressed to 'Dear CEO'"


with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)


